# Fetch Codebase and Models

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'higan'
!git clone https://github.com/genforce/higan.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')
!mkdir -p models/pretrain/pytorch
!wget https://www.dropbox.com/s/h1w7ld4hsvte5zf/stylegan_bedroom256_generator.pth?dl=1 -O models/pretrain/pytorch/stylegan_bedroom256_generator.pth --quiet
!wget https://www.dropbox.com/s/hwjyclj749qtp89/order_w.npy?dl=1 -O order_w_1k.npy --quiet


# Define Utility Functions

In [ ]:
import io
import IPython.display
import cv2
import PIL.Image
import os
import numpy as np
from tqdm import tqdm
import torch

from models.helper import build_generator
from utils.logger import setup_logger
from utils.editor import get_layerwise_manipulation_strength
from utils.editor import manipulate


def imshow(images, col, viz_size=256):
  """Shows images in one figure."""
  num, height, width, channels = images.shape
  assert num % col == 0
  row = num // col

  fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

  for idx, image in enumerate(images):
    i, j = divmod(idx, col)
    y = i * viz_size
    x = j * viz_size
    if height != viz_size or width != viz_size:
      image = cv2.resize(image, (viz_size, viz_size))
    fused_image[y:y + viz_size, x:x + viz_size] = image

  fused_image = np.asarray(fused_image, dtype=np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(fused_image).save(data, 'jpeg')
  im_data = data.getvalue()
  disp = IPython.display.display(IPython.display.Image(im_data))
  return disp

def build_model(model_name, logger=None):
  """Builds the generator by model name."""
  model = build_generator(model_name, logger=logger)
  return model

def sample_codes(model, num, seed=0, w1k_code=None):
  """Samples latent codes randomly."""
  np.random.seed(seed)
  if w1k_code is None:
    codes = generator.easy_sample(num)
    latent_codes = model.easy_sample(num=num, latent_space_type='w')
  else:
    latent_codes = w1k_code[np.random.randint(0, w1k_code.shape[0], num)] 
  latent_codes = model.easy_synthesize(latent_codes=latent_codes,
                                       latent_space_type='w',
                                       generate_style=False,
                                       generate_image=False)['wp']
  return latent_codes

w1k_code = np.load('order_w_1k.npy')

# Build Generator

In [ ]:
indoor_model_name = "stylegan_bedroom" 
indoor_model = build_model(indoor_model_name)

# Sample Codes

In [ ]:
#@title { display-mode: "form", run: "auto" }
num_samples = 4 #@param {type:"slider", min:1, max:8, step:1}
noise_seed = 382 #@param {type:"slider", min:0, max:1000, step:1}

indoor_latent_codes = sample_codes(indoor_model, num_samples, noise_seed, w1k_code=w1k_code)
synthesis_kwargs = {'latent_space_type': 'wp'}

images = indoor_model.easy_synthesize(indoor_latent_codes, **synthesis_kwargs)['image']
imshow(images, col=num_samples)

# Edit Scene Attributes

In [ ]:
#@title { display-mode: "form", run: "auto" }

attribute_name = 'view' #@param ['indoor_lighting', 'wood', 'cluttered_space', 'view']
path = f'boundaries/{indoor_model_name}/{attribute_name}_boundary.npy'
#print(f'Loading boundary from {path}')
try:
  boundary_file = np.load(path, allow_pickle=True).item()
  boundary = boundary_file['boundary']
  manipulate_layers = boundary_file['meta_data']['manipulate_layers']
except ValueError:
  boundary = np.load(path)
  if attribute_name == 'view':
    manipulate_layers = '0-4'
  else:
    manipulate_layers = '6-11'


if attribute_name == 'view':
  strength = [1.0 for _ in range(indoor_model.num_layers)]
else:
  strength = get_layerwise_manipulation_strength(
    indoor_model.num_layers, indoor_model.truncation_psi, indoor_model.truncation_layers)

distance = -3 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
indoor_codes = manipulate(latent_codes=indoor_latent_codes,
                     boundary=boundary,
                     start_distance=0,
                     end_distance=distance,
                     step=2,
                     layerwise_manipulation=True,
                     num_layers=indoor_model.num_layers,
                     manipulate_layers=manipulate_layers,
                     is_code_layerwise=True,
                     is_boundary_layerwise=False,
                     layerwise_manipulation_strength=strength)
images = indoor_model.easy_synthesize(indoor_codes[:, 1], latent_space_type='wp')['image']
imshow(images, col=num_samples)